In [64]:
# Cell type : CodeRead

import numpy as np
import matplotlib.pyplot as plt

pi=np.pi

In [304]:
# Cell type : CodeWrite

def gaussian(x,mu,Sigma):
    
    d = x.shape[0]
    a = -0.5
    const = (1/(np.sqrt(np.power(2*np.pi,d)*np.abs(np.linalg.det(Sigma)))))
    exp = np.exp(a * np.dot(  np.dot( (x-mu),np.linalg.inv(Sigma) ) ,np.transpose(x-mu)))
    
    return const*exp
    
def Bayes_A(X_train, Y_train, X_test):
    """ Give prediction for test instance using assumption BayesA.

    Arguments:
    X_train: numpy array of shape (n,d)
    Y_train: +1/-1 numpy array of shape (n,)
    X_test : numpy array of shape (m,d)

    Returns:
    Y_test_pred : +1/-1 numpy array of shape (m,)
    
    """
    n = X_train.shape[0]
    d = X_train.shape[1]
    m = X_test.shape[0]
    
    y_pos_count = 0
    y_neg_count = 0
    mu_ML_y_pos = np.zeros(d)
    mu_ML_y_neg = np.zeros(d)
    
    for i in range(n):
        if(Y_train[i]==1):          
            y_pos_count+=1
            mu_ML_y_pos += X_train[i,:]
        else:
            y_neg_count+=1
            mu_ML_y_neg += X_train[i,:]
            
    mu_ML_y_pos = mu_ML_y_pos/y_pos_count
    mu_ML_y_neg = mu_ML_y_neg/y_neg_count
        
    Py_1 = (y_pos_count)/(y_pos_count + y_neg_count)
        
    #print(y_pos_count,y_neg_count,mu_ML_y_pos,mu_ML_y_neg)
    #test
    I = np.eye(d)
    y_preds = np.zeros(m)
    for i in range(m):
        
        term_a = gaussian(X_test[i],mu_ML_y_pos,I)*Py_1
        term_b = gaussian(X_test[i],mu_ML_y_neg,I)*(1-Py_1)
        
        posterior_a  = (term_a)/(term_a+term_b)
        posterior_b  = (term_b)/(term_a+term_b)
        
        if( posterior_a >= posterior_b):
            y_preds[i] = 1
        else:
            y_preds[i] = -1
        
    return y_preds
        
    
    
    
def Bayes_B(X_train, Y_train, X_test):
    """ Give prediction for test instance using assumption BayesB.

    Arguments:
    X_train: numpy array of shape (n,d)
    Y_train: +1/-1 numpy array of shape (n,)
    X_test : numpy array of shape (m,d)

    Returns:
    Y_test_pred : +1/-1 numpy array of shape (m,)
    
    """
    n = X_train.shape[0]
    d = X_train.shape[1]
    m = X_test.shape[0]
    
    y_pos_count = 0
    y_neg_count = 0
    mu_ML_y_pos = np.zeros(d)
    mu_ML_y_neg = np.zeros(d)
    mu_ML_y     = np.zeros(d)
    
    for i in range(n):
        mu_ML_y += (X_train[i,:]/n)
        if(Y_train[i]==1):
            y_pos_count+=1
            mu_ML_y_pos += X_train[i,:]
        else:
            y_neg_count+=1
            mu_ML_y_neg += (X_train[i,:])
            
    mu_ML_y_pos = mu_ML_y_pos/y_pos_count
    mu_ML_y_neg = mu_ML_y_neg/y_neg_count
        
    #print(mu_ML_y_pos,mu_ML_y_neg)
    
    Py_1 = (y_pos_count)/(y_pos_count + y_neg_count)
        
    #test
    Sigma_ML_y = np.zeros((d,d))
    
    #print((X_train[0,:]-mu_ML_y).reshape(2,1),(np.transpose(X_train[0,:]-mu_ML_y).reshape(2,1)))
    #print((np.dot( np.transpose(X_train[0,:]-mu_ML_y) , (X_train[0,:]-mu_ML_y) )))
    
    for i in range(n):
        Sigma_ML_y += (np.dot( (np.transpose((X_train[i,:]-mu_ML_y).reshape(1,2))) , (X_train[i,:]-mu_ML_y).reshape(1,2) ))/n
        
#     print(mu_ML_y)
#     print(Sigma_ML_y)
        
    y_preds = np.zeros(m)
    for i in range(m):
        term_a = gaussian(X_test[i],mu_ML_y_pos,Sigma_ML_y)*Py_1
        term_b = gaussian(X_test[i],mu_ML_y_neg,Sigma_ML_y)*(1-Py_1)
        
        posterior_a  = (term_a)/(term_a+term_b)
        posterior_b  = (term_b)/(term_a+term_b)
        
        if( posterior_a >= posterior_b):
            y_preds[i] = 1
        else:
            y_preds[i] = -1
        
    return y_preds

def catog_indices(Y_train):
    lst_y_a=[] #stores indices of X's correspondint to y=-1
    lst_y_b=[] #stores indices of X's correspondint to y=+1
    cnt=0
    for i in Y_train:
        if i==-1:
            lst_y_a.append(cnt)
        if i==+1:
            lst_y_b.append(cnt)
        cnt=cnt+1
    return [lst_y_a,lst_y_b]

class Normal_Dist():            # PASSED
    def __init__(self,mu,sig):
        self.mu=mu
        self.sig=sig
        self.dim_in=len(mu) ## for further improvization, check whether dimensions of mu and sig are equal...
    
    def value(self,x):
        d=self.dim_in
        x_cen=x-self.mu
        sig_inv=np.linalg.inv(self.sig)
        x_cen=x_cen.reshape(1,2)
        val= np.sqrt(   ((2*pi)**(d))*(np.abs(np.linalg.det(self.sig)))   )
        val=(1/val)*( np.exp(-0.5*( x_cen @ sig_inv @ x_cen.transpose() )) )
        return val[0][0]

def Bayes_C(X_train, Y_train, X_test): ##########################################################################################
    """ Give prediction for test instance using assumption BayesC.

    Arguments:
    X_train: numpy array of shape (n,d)
    Y_train: +1/-1 numpy array of shape (n,)
    X_test : numpy array of shape (m,d)

    Returns:
    Y_test_pred : +1/-1 numpy array of shape (m,)
    
    """
    n = X_train.shape[0]
    d = X_train.shape[1]
    m = X_test.shape[0]
    
    y_pos_count = 0
    y_neg_count = 0
    mu_ML_y_pos = np.zeros(d)
    mu_ML_y_neg = np.zeros(d)
    mu_ML_y     = np.zeros(d)
    
    # for i in range(n):
    #     mu_ML_y += (X_train[i,:]/n)
    #     if(Y_train[i]==1):
    #         y_pos_count+=1
    #         mu_ML_y_pos += X_train[i,:]
    #     else:
    #         y_neg_count+=1
    #         mu_ML_y_neg += (X_train[i,:])
            
    # mu_ML_y_pos = mu_ML_y_pos/y_pos_count
    # mu_ML_y_neg = mu_ML_y_neg/y_neg_count
    
    [lst_y_a,lst_y_b]=catog_indices(Y_train)
    mu_ML_y_neg=X_train[lst_y_a].mean(axis=0)
    mu_ML_y_pos=X_train[lst_y_b].mean(axis=0)
    y_pos_count=len(lst_y_b)
    y_neg_count=len(lst_y_a)
    Py_1 = (y_pos_count)/(y_pos_count + y_neg_count)
    Py_2 = (y_neg_count)/(y_pos_count + y_neg_count)
        
    
    #test
    Sigma_ML_y_pos = np.zeros((d,d))
    Sigma_ML_y_neg = np.zeros((d,d))
    
    # for i in range(n):
        # if(Y_train[i]==1):
        #     Sigma_ML_y_pos += (np.dot( np.transpose((X_train[i,:]-mu_ML_y_pos).reshape(1,2)) , (X_train[i,:]-mu_ML_y_pos).reshape(1,2) ))
        # if(Y_train[i]==-1):
        #     Sigma_ML_y_neg += (np.dot( np.transpose((X_train[i,:]-mu_ML_y_neg).reshape(1,2)) , (X_train[i,:]-mu_ML_y_neg).reshape(1,2) ))
            

##
    for i in X_train[lst_y_a]:    # Estimating value of ∑-
        Sigma_ML_y_neg=Sigma_ML_y_neg + ( (i-mu_ML_y_neg).reshape(2,1) @ (i-mu_ML_y_neg).reshape(1,2) )

    for i in X_train[lst_y_b]:    # Estimating value of ∑+
        Sigma_ML_y_pos=Sigma_ML_y_pos + ( (i-mu_ML_y_pos).reshape(2,1) @  (i-mu_ML_y_pos).reshape(1,2) )

    Sigma_ML_y_pos = Sigma_ML_y_pos/y_pos_count
    Sigma_ML_y_neg = Sigma_ML_y_neg/y_neg_count

##
        
#     print(mu_ML_y)
#     print(Sigma_ML_y_pos,Sigma_ML_y_pos)
        
    y_preds=np.zeros(X_test.shape[0])

    for i in range(X_train.shape[0]):
        term_a = gaussian(X_test[i],mu_ML_y_pos,Sigma_ML_y_pos)*Py_1
        term_b = gaussian(X_test[i],mu_ML_y_neg,Sigma_ML_y_neg)*(Py_2)
        
        if( term_a  >= term_b):
            y_preds[i] = 1
        else:
            y_preds[i] = -1
        
    return y_preds


    NDa=Normal_Dist(mu_ML_y_neg,Sigma_ML_y_neg)
    NDb=Normal_Dist(mu_ML_y_pos,Sigma_ML_y_pos)

    for i in range( X_train.shape[0] ):
        if ( (1-Py_1)*NDa.value(X_train[i]) > Py_1*NDb.value(X_train[i]) ):
            y_preds[i]=-1
        if ( (1-Py_1)*NDa.value(X_train[i]) <= Py_1*NDb.value(X_train[i]) ):
            y_preds[i]=+1
    return y_preds


In [305]:

Y_test_predAa=Bayes_A(lsta[0],lsta[1],lsta[2])
Y_test_predAb=Bayes_A(lstb[0],lstb[1],lstb[2])   # Running all datasets on BayesA
Y_test_predAc=Bayes_A(lstc[0],lstc[1],lstc[2])
# Y_test_predA=[Y_test_predAa,Y_test_predAb,Y_test_predAc]

Y_test_predBa=Bayes_B(lsta[0],lsta[1],lsta[2])
Y_test_predBb=Bayes_B(lstb[0],lstb[1],lstb[2])   # Running all datasets on BayesB
Y_test_predBc=Bayes_B(lstc[0],lstc[1],lstc[2])
# Y_test_predB=[Y_test_predBa,Y_test_predBb,Y_test_predBc]

Y_test_predCa=Bayes_C(lsta[0],lsta[1],lsta[2])
Y_test_predCb=Bayes_C(lstb[0],lstb[1],lstb[2])   # Running all datasets on BayesC
Y_test_predCc=Bayes_C(lstc[0],lstc[1],lstc[2])
# Y_test_predC=[Y_test_predCa,Y_test_predCb,Y_test_predCc]

errAa = (0.5*(abs((Y_test_predAa-lsta[3])).sum())/len(Y_test_predAa))*100
errAb = (0.5*(abs((Y_test_predAb-lstb[3])).sum())/len(Y_test_predAb))*100
errAc = (0.5*(abs((Y_test_predAc-lstc[3])).sum())/len(Y_test_predAc))*100

errBa = (0.5*(abs((Y_test_predBa-lsta[3])).sum())/len(Y_test_predBa))*100 #
errBb = (0.5*(abs((Y_test_predBb-lstb[3])).sum())/len(Y_test_predBb))*100
errBc = (0.5*(abs((Y_test_predBc-lstc[3])).sum())/len(Y_test_predBc))*100

errCa = (0.5*(abs((Y_test_predCa-lsta[3])).sum())/len(Y_test_predCa))*100 #
errCb = (0.5*(abs((Y_test_predCb-lstb[3])).sum())/len(Y_test_predCb))*100
errCc = (0.5*(abs((Y_test_predCc-lstc[3])).sum())/len(Y_test_predCc))*100

print(errAa,errBa,errCa)
print(errAb,errBb,errCb)
print(errAc,errBc,errCc)

9.8 22.900000000000002 22.55
50.849999999999994 50.4 7.449999999999999
11.75 11.65 11.799999999999999


In [306]:
# 9.8 22.900000000000002 22.55
# 50.849999999999994 50.4 7.449999999999999
# 11.75 11.65 11.799999999999999

In [307]:
Y_train = lsta[1]
X_train = lsta[0]
[lst_y_a,lst_y_b]=catog_indices(Y_train)
mu_ML_y_neg=X_train[lst_y_a].mean(axis=0)
mu_ML_y_pos=X_train[lst_y_b].mean(axis=0)
y_pos_count=len(lst_y_b)
y_neg_count=len(lst_y_a)
Py_1 = (y_pos_count)/(y_pos_count + y_neg_count)
Py_2 = (y_neg_count)/(y_pos_count + y_neg_count)

d=X_train.shape[1]
Sigma_ML_y_pos = np.zeros((d,d))
Sigma_ML_y_neg = np.zeros((d,d))

for i in X_train[lst_y_a]:    # Estimating value of ∑-
    Sigma_ML_y_neg=Sigma_ML_y_neg + ( (i-mu_ML_y_neg).reshape(2,1) @ (i-mu_ML_y_neg).reshape(1,2) )

for i in X_train[lst_y_b]:    # Estimating value of ∑+
    Sigma_ML_y_pos=Sigma_ML_y_pos + ( (i-mu_ML_y_pos).reshape(2,1) @  (i-mu_ML_y_pos).reshape(1,2) )

Sigma_ML_y_pos = Sigma_ML_y_pos/y_pos_count
Sigma_ML_y_neg = Sigma_ML_y_neg/y_neg_count


y_preds=np.zeros(X_test.shape[0])

for i in range(X_train.shape[0]):
    term_a = gaussian(X_test[i],mu_ML_y_pos,Sigma_ML_y_pos)*Py_1
    term_b = gaussian(X_test[i],mu_ML_y_neg,Sigma_ML_y_neg)*(Py_2)
    if( term_a  >= term_b):
        y_preds[i] = 1
    else:
        y_preds[i] = -1

# NDa=Normal_Dist(mu_ML_y_neg,Sigma_ML_y_neg)
# NDb=Normal_Dist(mu_ML_y_pos,Sigma_ML_y_pos)
# for i in range( X_train.shape[0] ):
#     if ( (1-Py_1)*NDa.value(X_train[i]) > Py_1*NDb.value(X_train[i]) ):
#         y_preds[i]=-1
#     if ( (1-Py_1)*NDa.value(X_train[i]) <= Py_1*NDb.value(X_train[i]) ):
#         y_preds[i]=+1


In [308]:
sam_test=X_train[1979]
print("for -1", Py_2*NDa.value(sam_test))
print("for +1", Py_1*NDb.value(sam_test))
print("for -1", Py_2*gaussian(sam_test,mu_ML_y_neg,Sigma_ML_y_neg))
print("for +1", Py_1*gaussian(sam_test,mu_ML_y_pos,Sigma_ML_y_pos))

for -1 0.2825661240250485
for +1 1.050983601169713e-26
for -1 0.2825661240250485
for +1 1.050983601169713e-26


In [309]:
Py_1*gaussian(sam_test,mu_ML_y_pos,Sigma_ML_y_pos) == Py_1*NDb.value(sam_test)


True

In [310]:
a=2